In [3]:
# Masking code
import torch
import numpy as np
def mask_gen(epoch, tile_sz=4, image_sz=64, start=15, end=85, lower = 80, upper=100):
    assert image_sz%tile_sz == 0    #code not tested for other cases
    mask_ratio = 0.01*min(upper, max(lower, lower + ((upper-lower)*(epoch-start))/(end-start)))
    iv,jv = [x.flatten() for x in np.meshgrid(np.arange(image_sz//tile_sz), np.arange(image_sz//tile_sz), indexing='ij')]
    mask_i = np.random.choice(len(iv), int(mask_ratio*len(iv)), replace=False)

    mask = torch.ones((image_sz,image_sz))
    for idx in zip(mask_i):
        i, j = iv[idx]*tile_sz, jv[idx]*tile_sz
        mask[i:i+tile_sz,j:j+tile_sz] = 0
    print(mask_ratio, (mask == 0).sum() / len(mask.flatten()))
    return mask
    
mask_gen(50)

0.9 tensor(0.8984)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [4]:
#Postiional encoding
import torch
from positional_encodings.torch_encodings import PositionalEncoding1D

def get_pos_emb(tstep, xx_len, test_mode=1, w=64, h=64):
    inp_len = xx_len // 2

    def single_channel(tstep, inp_len, dim, b_size=1):
        # b_size is irrelavant, the value will be same for each batch dimension
        p_enc_1d_model = PositionalEncoding1D(dim)
        emb_full =  p_enc_1d_model(torch.rand(b_size, tstep + inp_len, dim))
        return emb_full[:,-inp_len:]

    single_ch_emb = single_channel(tstep, inp_len, h*w, 1)
    # we want concatenation to be alternate: https://stackoverflow.com/questions/61026393/pytorch-concatenate-rows-in-alternate-order
    # After operation, emb[0,i] == emb[0,i+1] where i is even
    emb = torch.cat((single_ch_emb, single_ch_emb), axis=-1).reshape((1, -1, h, w))  
    if test_mode > 1:
        emb = [emb for _ in range(test_mode)]
        emb = torch.cat(emb, dim=-1)
    return emb

emb = get_pos_emb(2, 62, test_mode=7)
    

ModuleNotFoundError: No module named 'positional_encodings'